## Kaggle - Natural Language Processing with Disaster Tweets
Solution by Chris & Naru 

https://www.kaggle.com/learn/natural-language-processing

### Useful Resources
https://pandas.pydata.org/docs/

https://towardsdatascience.com/how-to-combine-textual-and-numerical-features-for-machine-learning-in-python-dc1526ca94d9

In [1]:
import pandas as pd
file_loc = "data\\"

df = pd.read_csv(file_loc + "train.csv")

In [2]:
print(df.columns)
print("\n")
print(df.shape)

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')


(7613, 5)


In [3]:
def output_matrix(pred, actual):  
    """output_matrix computes accuracy information,
    and a confusion matrix"""
    
    df = pd.DataFrame({"pred":pred, "actual":actual, "count":0})
    # Rounds each predicted number to either 0 or 1.
    df.pred = df.pred.apply((lambda x : 0 if 0 >= round(x) else 1))
    
    correct = len(df[df.pred == df.actual])
    incorrect = len(df[df.pred != df.actual])
    accuracy = correct / (correct + incorrect)
    
    print(f"Correct: {correct}\nIncorrect: {incorrect}\nAccuracy: {accuracy}")

    print(df.groupby(["actual","pred"]).count())

In [4]:
# First of 5 disaster tweets and 5 non-disaster tweets
# 0 means non-disaster
# 1 means disaster
sample_df_0 = df[df.target==0].head(5)
sample_df_1 = df[df.target==1].head(5)
sample_df = sample_df_0.append(sample_df_1)

sample_df.reset_index(drop=True)

,id,keyword,location,text,target
0,23,NaN,NaN,What's up man?,0
1,24,NaN,NaN,I love fruits,0
2,25,NaN,NaN,Summer is lovely,0
3,26,NaN,NaN,My car is so fast,0
4,28,NaN,NaN,What a goooooooaaaaaal!!!!!!,0
5,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
6,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
7,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
8,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
9,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
df[["id","target"]].groupby("target").count().rename(columns={"id":"Count"})

,Count
target,
0,4342
1,3271


In [6]:
# Calculating the length of text
df["text_length"]=df["text"].str.len()
df[["target","text_length"]].groupby("target").describe()

text_length                                                        
             count        mean        std   min   25%    50%    75%    max
target                                                                    
0           4342.0   95.832105  35.969652   7.0  68.0  101.0  130.0  157.0
1           3271.0  108.236319  29.369513  14.0  88.0  115.0  136.0  151.0

In [7]:
from scipy.stats import ttest_ind
target_0 = df[df.target==0]["text_length"]
target_1 = df[df.target==1]["text_length"]
ttest_ind(target_0, target_1, equal_var=False, alternative="less")



Ttest_indResult(statistic=-16.551040413805307, pvalue=9.029679548959969e-61)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
wordCounts = {}

for x in df.text:
    for y in x.split(" "):
        
        word = y.lower().replace("?","").replace(".","")
        
        setVal = wordCounts.get(y.lower(),0) + 1
        wordCounts[y.lower()] = setVal

popularWords = list(filter(lambda x: (x[1] > 20), wordCounts.items()))
popularWords = [x[0] for x in popularWords]
len(popularWords)

752

In [26]:
tfVect = TfidfVectorizer(vocabulary = popularWords)


test2 = tfVect.fit_transform(df["text"])
ndf = pd.DataFrame(test2.toarray(), columns = tfVect.get_feature_names())

ndf["text_length"] = df.text_length


In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(ndf, df.target, test_size=0.5, random_state=42)

lRegModel = LinearRegression()

lRegModel.fit(x_train, y_train)

predictionResult = lRegModel.predict(x_test)

output_matrix(predictionResult, y_test)

Correct: 2947
Incorrect: 860
Accuracy: 0.774100341476228
             count
actual pred       
0      0      1821
       1       364
1      0       496
       1      1126


In [28]:
from sklearn.ensemble import RandomForestClassifier

x_train, x_test, y_train, y_test = train_test_split(ndf, df.target, test_size=0.5, random_state=42)

rfcModel = RandomForestClassifier()

rfcModel.fit(x_train, y_train)

predictionResult = rfcModel.predict(x_test)

predictionResult

output_matrix(predictionResult, y_test)

Correct: 2928
Incorrect: 879
Accuracy: 0.7691095350669819
             count
actual pred       
0      0      1925
       1       260
1      0       619
       1      1003


In [29]:
from sklearn.naive_bayes import MultinomialNB

x_train, x_test, y_train, y_test = train_test_split(ndf, df.target, test_size=0.5, random_state=42)

mnbModel = MultinomialNB()

mnbModel.fit(x_train, y_train)

predictionResult = mnbModel.predict(x_test)

predictionResult

output_matrix(predictionResult, y_test)

Correct: 2971
Incorrect: 836
Accuracy: 0.7804045179931705
             count
actual pred       
0      0      1924
       1       261
1      0       575
       1      1047
